# Introduction

In [1]:
from ipywebcam import WebCamWidget, WebCamRecorder, Record, RecordPlayer, FileListFactory

In [2]:
factory = FileListFactory("test", "$i4-$Y-$m-$d-$H-${M}-${S}-$f-$uh6.mp4", condition=FileListFactory.create_frame_based_condition(100))

In [3]:
w = WebCamWidget(
    iceServers=[{"urls": "turn:190.92.221.222", "username": "admin", "credential": "123456"}],
    constraints={
        "video": {
            "frameRate": { "max": 10 },
            "width": 256,
            "height": 192,
        },
    },
)
w

WebCamWidget(constraints={'video': {'frameRate': {'max': 10}, 'width': 256, 'height': 192}}, iceServers=[{'url…

In [4]:
recorder = WebCamRecorder(w, factory)
recorder.start()

load 3 records


In [5]:
import cv2
from av import VideoFrame
async def transform(frame, ctx):
    img = frame.to_ndarray(format="bgr24")
    # prepare color
    img_color = cv2.pyrDown(cv2.pyrDown(img))
    for _ in range(6):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
    img_color = cv2.pyrUp(cv2.pyrUp(img_color))

    # prepare edges
    img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_edges = cv2.adaptiveThreshold(
        cv2.medianBlur(img_edges, 7),
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        9,
        2,
    )
    img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

    # combine color and edges
    img = cv2.bitwise_and(img_color, img_edges)

    # rebuild a VideoFrame, preserving timing information
    new_frame = VideoFrame.from_ndarray(img, format="bgr24")
    # new_frame.pts = frame.pts
    # new_frame.time_base = frame.time_base
    return new_frame

transformer = w.add_video_transformer(transform)

In [6]:
w.remove_video_transformer(transformer)

In [7]:
recorder.stop()

In [8]:
w.close_peers()

In [8]:
player = RecordPlayer(recorder=recorder)
player

RecordPlayer()